# Notas sobre la información

### Empezamos por analizar los datos. Tenemos el ID personal, el genero de la persona, la edad de la persona, si tiene hipertensión, si tiene enfermedades cardiacas, si ha estado casado, su tipo de trabajo, su tipo de vivienda, su promedio de glucosa sanguinea, su IMC, su experiencia como fumador, y si ha tenido derrames.

### Tenemos los datos representados de distinitas formas, el ID es un número entero, el sexo es un string, la edad es un float, hypertensión es un binario que indica verdadero o falso, lo mismo con enfermedades del corazón, si esta casado o no es un string, su tipo de trabajo y su tipo de residencia son string, el nivel de glucosa es un entero con decimales, el IMC es un entero con decimales, su experiencia con el cigarro es string y si ha tenido o no derrames es un binario.

### A partir de lo que tenemos queremos hacer un modelo de ML que sea capaz de predecir si una persona esta en riesgo de sufrir un derrame. Queremos saber cuales son las razones que influyen en ese riesgo y a partir de eso hacer una conclusión que nos diga que comportamientos generan o aumentan la probabilidad de derrames, y poder informar al público sobre esto.

# Empezamos con el código.

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [5]:
#Importamos los datos
data = pd.read_csv("healthcare-dataset-stroke-data.csv")
data.info()

FileNotFoundError: [Errno 2] No such file or directory: 'healthcare-dataset-stroke-data.csv'

### Vemos los tipos de datos y sí tenemos datos vacios dentro de las columnas. En total tenemos datos de 5110 personas, en la columna de datos de IMC tenemos datos vacios, por lo que tenemos que resolver. Vamos a eliminar todas las filas en las que IMC tenga dato vacio.

In [3]:
data = data.dropna(subset = ['bmi'])
data.info()

NameError: name 'data' is not defined

In [ ]:
atributos_num1 = data[['age', 'avg_glucose_level', 'bmi', 'stroke', 'gender']]
atributos_personales = data[["gender", "ever_married", "work_type", "Residence_type", "smoking_status"]]

In [ ]:
sns.set_style("darkgrid")
g = sns.pairplot(atributos_num1, diag_kind="kde")
g.map_lower(sns.kdeplot, levels = 5, color = "#F0A531FF", )

### En estas graficas podemos observar las relaciones que tienen las variables y la distribución de los datos por variable. Los histogramas representan la distribucion y son más faciles de visualizar, donde sea mayor se condensan más personas, y los diagramas de puntos son todos los datos disponibles, cada punto representa una persona.
### Tambien notamos que la mayoria de las graficas no son lineares, y tienden a dividirse en dos partes. Una de ellos conteniendo la mayoria de las personas por una gran diferencia, la unica grafica que no es así es la grafica de edad lo que representa que tenemos personas de todas las edades en algo que parece una distribuición de campana.

### Las lines naranjas representan en donde se concentran más los datos separados en 5 niveles, es decir cada 20% de los datos es una linea.

In [ ]:
g = sns.pairplot(atributos_num1, diag_kind="kde", hue = "stroke")

In [ ]:
g = sns.pairplot(atributos_num1, diag_kind="kde", hue = "gender")

### Coloreando a las personas que han sufrido de derrames es más fácil identificar sus caracteristicas y sus datos, con esto ya nos podemos empezar a dar una idea, este es solo el primer paso, para asegurarnos vamos a utilizar un "heat-map" y ver cual es el parametro que más relación tiene con un derrame.

In [ ]:
plt.figure(figsize=(10,8))
relacion = atributos_num1.corr()
sns.heatmap(relacion, annot=True, cmap=plt.cm.Blues)
plt.show()

### Con este mapa se observa que la edad es posiblemente el mayor influyente para un derrame, ya que sus cuadros son los cuadros con más relación y el que menos influencia tenga sea el IMC.

In [ ]:
data["gender"].value_counts().plot(kind='bar')
plt.title("gender")
plt.grid()
plt.show()
print(data["gender"].value_counts())

### Vemos que el valor de "Other" no es muy significativo, por lo que lo eliminamos para simplificar el trabajo.

In [ ]:
data["stroke"].value_counts().plot(kind='bar')
plt.title("Stroke")
plt.grid()
plt.show()
print(data["stroke"].value_counts())

### Vemos que tenemos muy pocas personas que han sufrido de derrames, por lo que también tenemos pocos datos para ellos y eso puede afectar cuando hagamos nuestro modelo. 

In [ ]:
data["smoking_status"].value_counts().plot(kind='bar')
plt.title("Smoking Status")
plt.grid()
plt.show()
print(data["smoking_status"].value_counts())

In [ ]:
data["work_type"].value_counts().plot(kind='bar')
plt.title("Work Type")
plt.grid()
plt.show()
print(data["work_type"].value_counts())

In [ ]:
data.drop(data.loc[data['gender']== 'Other'].index, inplace=True)

In [ ]:
data.drop(data.loc[data['work_type']== 'Never_worked'].index, inplace=True)
data.info()

In [ ]:
bmi = sns.stripplot(x="stroke", y="bmi", data = data, jitter=0.25)

### Queremos ver sí el IMC tiene algún patrón pero vemos que los que tienen derrames, se encuentran dentro de la mayoria de la población.

In [ ]:
bmi = sns.stripplot(x="stroke", y="age", data = data, jitter=0.25)

### Con esta grafica confirmamos que la edad es un fuerte determinante para un derrame, ya que la mayoria de la población se encuentra más alta y muy pocos puntos se encuentran bajos.

In [ ]:
bmi = sns.stripplot(x="stroke", y="avg_glucose_level", data = data, jitter=0.25)

### Este puede ser un poco relevante pero no tan fuerte como la edad, en este caso vemos más puntos en la parte baja de la grafica pero no es muy significativo. 
### Vamos a sacar valores generales de los datos.

In [ ]:
DataFrame = pd.DataFrame(data)
DataFrame.pop("id")
DataFrame.pop("gender")
DataFrame.pop("ever_married")
DataFrame.pop("work_type")
DataFrame.pop("smoking_status")
DataFrame.pop("Residence_type")
DataFrame.info()

In [ ]:
DataFrame.skew(axis = 0)
#Simetría de los datos, mientras más cerca del 0 más simetrico es.

In [ ]:
DataFrame.kurtosis(axis = 0)
#Concentración de los valores en torno a su media. Mientras más alejado del 0 menos concentrado es.

In [ ]:
DataFrame.std(axis = 0)
#Que tan dispersos son los valores de la media, mientras más alto el número más dispersos son.

In [ ]:
DataFrame.var(axis=0)
#Varianza, representa la variabilidad de los datos conforme a su media.

In [ ]:
DataFrame.median(axis=0)
#El valor que parte los datos en el medio.

In [ ]:
DataFrame.mean(axis=0)
#El promedio de los datos

### A partir de todos estos calculos y junto con las primeras graficas del proyecto nos podemos dar una idea mucho mayor de los datos, por ejemplo, podemos observar que tenemos mucha diferencia en las poblaciones con problemas de corazón y en la de personas con derrames. Otro es que los niveles de glucosa son los datos mas variados que tenemos junto con los de edad, la edad media es de 42 años, casi no tenemos personas con hypertension, problemas del corazón o derrames, el nivel de glucosa promedio es de 105.33, y el promedio de IMC es de 28.

### Mucha de esta información ya la teniamos pero de estos calculos adquirimos mucha nueva información.

### Empezamos a hacer el modelo de ML. Vamos a utilizar un modelo de Random Forest, que es una forma de ML que nos permite predecir con mayor precision un resultado ya que analiza varias posibilidades de modelos y los combina para predecir el mejor resultado. 

In [ ]:
parametros = ['gender', 'hypertension', 'heart_disease', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']
df = data
data_feature = pd.get_dummies(df, columns = parametros)
data_feature.head()

In [ ]:
data_feature.pop('id')
Final = data_feature
Final.head()

### Estos son los datos que vamos a tomar en cuenta para el modelo para el modelo. Hicimos este proceso para que las opciones de cada dato sea un sí o un no o datos numericos, no podemos trabajar con datos que no sean numericos.Esta es nuestra caja de herramientas de donde vamos a sacar valores aleatoriamentes y generar varios modelos. 

In [ ]:
X = Final[[ 'age', 'avg_glucose_level', 'bmi', 'gender_Female',
            'gender_Male', 'hypertension_0', 'hypertension_1',
            'heart_disease_0', 'heart_disease_1', 'ever_married_No',
            'ever_married_Yes', 'work_type_Govt_job', 'work_type_Private',
            'work_type_Self-employed', 'work_type_children', 'Residence_type_Rural',
            'Residence_type_Urban', 'smoking_status_Unknown',
            'smoking_status_formerly smoked', 'smoking_status_never smoked',
            'smoking_status_smokes'
        ]]

In [ ]:
X.shape

### Tenemos un cuadro de datos de 4908 filas y 21 columnas. Lo que significa 4908 personas y 21 parametros por persona.

In [ ]:
y = Final[['stroke']]

In [ ]:
#https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
#https://www.youtube.com/watch?v=J4Wdy0Wc_xQ
X = preprocessing.StandardScaler().fit(X).transform(X)
X[0:3]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

### Creamos subconjuntos de datos aleatorios a partir de la base de datos general que usaremos para entrenar los arboles.

In [ ]:
weights = np.array([21 if i == 1 else 1 for i in y_train["stroke"]])
#intentamos balancer el peso de los datos

In [ ]:
FOREST = RandomForestClassifier(random_state = 3, n_estimators = 400, max_depth = 10).fit(X_train, y_train.values.ravel(), sample_weight = weights.T)

FOREST_TRAIN = FOREST.score(X_train, y_train)
FOREST_TEST = FOREST.score(X_test, y_test)

print('Precisión de subdata:\t', FOREST_TRAIN)
print('Precisión real:\t\t', FOREST_TEST)

In [ ]:
print("REPORTE DE RESULTADOS DE TRAIN---------")
print(classification_report(y_train, FOREST.predict(X_train)))
print()
print("REPORTE DE RESULTADOS DE TEST----------")
print(classification_report(y_test, FOREST.predict(X_test)))


# Modelado con Gradient Boosting

Quisimos hacer un acercamiento ha este tipo de algoritmo ya que promete ofrecer, aplicando dentro de los parametros correctos, una precision que pudiera mejorar la presicion que conseguimos utilizando Random Forest. 

Comenzamos importando las librerarias necesarias para utilizar el modelo GradientBoostingRegressor de Sklearn




In [ ]:
from sklearn.ensemble import GradientBoostingRegressor 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error 
from sklearn.metrics import r2_score
import warnings 
warnings.filterwarnings('ignore')




##

In [ ]:
#Create gradientboost REGRESSOR object
gradientregressor = GradientBoostingRegressor(max_depth=2,n_estimators=3, learning_rate=1.0)

In [ ]:
#Train gradientboost REGRESSOR
model = gradientregressor.fit(X_train, y_train)

#Predict the response of the dataset
y_pred = model.predict(x_test)


In [ ]:
r2_score(y_pred,y_test)


Aquí podemos ver el porcentaje de precisión que obtuvimos únicamente utilizando datos arbitrarios 

## Utilizando Hyperparameters Tuning


In [ ]:
rom sklearn.model_selection import GridSearchCV
LR = {'learning_rate': [0.15,0.1,0.10,0.05,0.20,0.18,0.17,0.11,0.12], 'n_estimators':[100,150,200,250,120,110,112,180,160,220,210]}

tuning = GridSearchCV(estimator =GradientBoostingRegressor(),param_grid = LR, scoring = 'r2')
tuning.fit(X_train,y_train)
tuning.best_params_, tuning.best_score_